<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Práctica 3 - Enumerativos </h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>
</div>

En ésta práctica utilizaremos el concepto de ramificación y poda para la resolución del siguiente problema.

La construcción de un artefacto requiere ensamblar $M$ piezas que identiﬁcamos con números entre $1$ y $M$. El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas. Los costes, que son valores positivos, se nos proporcionan en una matriz $C$. La celda $c_{ij}$ es el coste de ensamblar la pieza $i$ cuando ya se han ensamblado $j$ piezas. Deseamos calcular el orden de ensamblado de menor coste total, entendido éste como la suma del coste de ensamblaje de cada una de las piezas. 

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>
    </p></div>




<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>Ramificación y poda: Problema del ensamblaje</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
  Implementa mediante ramificación y poda una solución al problema del ensamblaje con tal de encontrar el listado ordenado de piezas cuyo coste total de ensamblaje sea mínimo.
  </p>
  </div>
    </div></div>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center> 

<p>
<h3>INPUT</h3>
<ul>
<li>C: Matriz de costes de tamaño $M\times M$, donde M es el número de piezas. El elemento $i,j$ de la matriz representa el coste de ensamblaje para la pieza $i$ tras haber colocado $j$ piezas. 
</li>
</ul>
<br>
<h3>OUTPUT</h3>
Lista de números de $1$ a $M$ ordenados de forma que el coste total de ensamblaje sea mínimo. Ej: [5, 3, 2, 4, 1]
</p>

</div>

In [30]:
import heapq
import numpy as np
#init
def branch_bound(C,M):
    """
    Función que obtiene el orden óptimo de las piezas tal que el coste total es mínimo.
    Args:
        - C: matriz de costes M x M
    """
    #matriz donde podremos seleccionar correctamente los items
    matriz=np.empty(shape=(M,M),dtype=bool)
    matriz.fill(True)

    cota_sup=cota_superior(C)
    heap=[ ] #cua de prioritats
    coste_acumulado=0 #el coste que acumulamos en cada ramificacion y tenemos el coste de la rama actual
    items= [float('inf')]*M #conjunt d'items resultant
    millor_solucio=[] 
    v=[]
    elem=0
    ctr=0
    #inicialitzem la cua
    for i in range(M):
        cost=get_cost(C,M,elem,i,v.copy())#minimo fila + coste objeto
        coste_acumulado=C[elem][i] #coste de la rama, se acumulará posteriormente
        it_private=items.copy()
        it_private[i]=elem
        matriz_n=matriz.copy()
        matriz_n=fill_matrix(M,i,elem,matriz_n)
        ctr+=1
        heap.append((cost,coste_acumulado,elem,it_private,i,ctr,matriz_n))
        coste_acumulado=0
        
    heapq.heapify(heap)
    
    while heap:
        actual, acc, elemento, its, col,_ctr,mat = heapq.heappop(heap)
        if actual>cota_sup:
            continue
        else:
            _=elemento+1
            if float('inf') not in its and actual<cota_sup: # si hay alguna mejor
                millor_solucio=its
                cota_sup=acc
            else:
                for q in range(M):
                    if mat[_][q]:
                        _acc=C[_][q]+acc
                        _actual=get_cost(C,M,_,q,v.copy())+_acc
                        _its=its.copy()
                        _its[q]=_
                        _mat=mat.copy()
                        _mat=fill_matrix(M,q,_,_mat)
                        ctr+=1
                        heapq.heappush(heap,(_actual,_acc,_,_its,ctr,q,_mat))
    return millor_solucio,cota_sup
                    


def get_cost(C,M,obj,col,visited):
    cost=C[obj][col] 
    j_temporal=0
    for i in range(obj+1,len(C)):
        less=float('inf')
        for j in range(len(C)):
            if j not in visited:
                if C[i][j]<less and j!=col:
                    less=C[i][j]
                    j_temporal=j
        visited.append(j_temporal)
        cost+=less
    return cost                
              
            
def fill_matrix(M,t,elem,matrix):
    for i in range(M):
        matrix[elem][i]=False
        matrix[i][t]=False
        
    return matrix

#Cogemos el menor de cada fila
def cota_inferior(C):
    inferior=0
    for i in C.keys():
        less=min(C[i].values())
        inferior+=less
    return inferior
#Cogemos los valores de la diagonal
def cota_superior(C):
    superior=0
    for i in C.keys():
        diag=C[i][i]
        superior+=diag
    return superior

In [31]:
import random
from random import randint
random.seed(0)
M = 10 # Número de piezas
# Matriz de costes generada aleatoriamente
C = {}
for i in range(M):
    C[i] = {}
    for j in range(M):
        C[i][j] = randint(1, 25)
        
    print(i,C[i])
#print(C)




solucion = branch_bound(C ,10)
print(solucion)


0 {0: 13, 1: 25, 2: 14, 3: 2, 4: 9, 5: 17, 6: 16, 7: 13, 8: 10, 9: 16}
1 {0: 12, 1: 19, 2: 7, 3: 17, 4: 5, 5: 10, 6: 5, 7: 25, 8: 4, 9: 20}
2 {0: 9, 1: 18, 2: 23, 3: 20, 4: 5, 5: 10, 6: 4, 7: 24, 8: 3, 9: 22}
3 {0: 11, 1: 16, 2: 18, 3: 4, 4: 12, 5: 14, 6: 11, 7: 20, 8: 21, 9: 7}
4 {0: 18, 1: 16, 2: 15, 3: 17, 4: 9, 5: 2, 6: 18, 7: 1, 8: 3, 9: 24}
5 {0: 13, 1: 23, 2: 22, 3: 21, 4: 1, 5: 20, 6: 16, 7: 11, 8: 8, 9: 24}
6 {0: 11, 1: 23, 2: 3, 3: 7, 4: 19, 5: 8, 6: 8, 7: 5, 8: 18, 9: 15}
7 {0: 3, 1: 3, 2: 11, 3: 17, 4: 16, 5: 4, 6: 10, 7: 18, 8: 10, 9: 23}
8 {0: 4, 1: 18, 2: 11, 3: 18, 4: 7, 5: 20, 6: 18, 7: 19, 8: 10, 9: 15}
9 {0: 3, 1: 20, 2: 13, 3: 11, 4: 19, 5: 8, 6: 10, 7: 6, 8: 7, 9: 6}
([8, 7, 6, 0, 5, 4, 1, 9, 2, 3], 36)


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios </h3></p> </center> 

</div>

### En esta sección debéis responder a las preguntas siguientes:
#### 1. Explica cómo has realizado tu implementación, enfatizando en la heurística empleada.
#### 2. Análisis de complejidad de la implementación

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>

<p>
La entrega tiene que consistir en un solo fichero **.ipynb** con el nombre: *Apellido1_Apellido2*_*Nombre*_ASS3.ipyn
</p>
</div>